In [38]:
import pandas as pd
loans_2007 = pd.read_csv("https://dsserver-prod-resources-1.s3.amazonaws.com/133/loans_2007.csv?versionId=VFO5xSKetnKGEdwAU1.uX1K.Y4Q07Xjg")
loans_2007.drop_duplicates()

print(loans_2007.iloc[0])
print(loans_2007.shape[1])

id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

Following features need to be removed:

* id: randomly generated field by Lending Club for unique identification purposes only
* member_id: also a randomly generated field by Lending Club for unique identification purposes only
* funded_amnt: leaks data from the future (after the loan is already started to be funded)
* funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
* grade: contains redundant information as the interest rate column (int_rate)
* sub_grade: also contains redundant information as the interest rate column (int_rate)
* emp_title: requires other data and a lot of processing to potentially be useful
* issue_d: leaks data from the future (after the loan is already completed funded)

While the grade and sub_grade values are categorical, the int_rate column contains continuous values, which are better suited for machine learning.

In [39]:
loans_2007 = loans_2007.drop(["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d"], axis=1)

Drop the following columns:

* zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
* out_prncp: leaks data from the future, (after the loan already started to be paid off)
* out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
* total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
* total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
* total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)

The out_prncp and out_prncp_inv both describe the outstanding principal amount for a loan, which is the remaining amount the borrower still owes. These 2 columns as well as the total_pymnt column describe properties of the loan after it's fully funded and started to be paid off. This information isn't available to an investor before the loan is fully funded and we don't want to include it in our model.

In [40]:
loans_2007 = loans_2007.drop(["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"], axis=1)

Drop the following columns:

* total_rec_int: leaks data from the future, (after the loan already started to be paid off),
* total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
* recoveries: also leaks data from the future, (after the loan already started to be paid off),
* collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
* last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
* last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).

All of these columns leak data from the future, meaning that they're describing aspects of the loan after it's already been fully funded and started to be paid off by the borrower.



In [41]:
loans_2007 = loans_2007.drop(["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1)
print(loans_2007.iloc[0])
print(loans_2007.shape[1])

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status                     f
last_credit_pull_d               J

In [42]:
print(loans_2007['loan_status'].value_counts())

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64


* Fully Paid: Loan has been fully paid off.
* Charged Off: Loan for which there is no longer a reasonable expectation of further payments.
* Does not meet the credit policy. Status:Fully Paid: While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
* Does not meet the credit policy. Status:Charged Off: While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
* In Grace Period: The loan is past due but still in the grace period of 15 days.
* Late (16-30 days): Loan hasn't been paid in 16 to 30 days (late on the current payment).
* Late (31-120 days): Loan hasn't been paid in 31 to 120 days (late on the current payment).
* Current: Loan is up to date on current payments.
* Default: Loan is defaulted on and no payment has been made for more than 121 days.

Only the Fully Paid and Charged Off values describe the final outcome of the loan. The other values describe loans that are still on going and where the jury is still out on if the borrower will pay back the loan on time or not.

In [43]:
loans_2007 = loans_2007[(loans_2007['loan_status'] == "Fully Paid") | (loans_2007['loan_status'] == "Charged Off")]

status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}

loans_2007 = loans_2007.replace(status_replace)

Remove columns that contain only one unique value

In [44]:
orig_columns = loans_2007.columns
drop_columns = []

for col in orig_columns:
    # Drop any null for each column & return unique values
    col_series = loans_2007[col].dropna().unique()
    # If there is only one unique value, append a column name into "drop_columns"
    if len(col_series) == 1:
        drop_columns.append(col)
        
loans_2007 = loans_2007.drop(drop_columns, axis=1)

print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


In [45]:
# Count the number of missing values

loans = loans_2007
null_counts = loans.isnull().sum()

print(null_counts[null_counts>0])

emp_length              1036
title                     11
revol_util                50
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


* Despite the large amount of missing values, employment length should be kept for assessing a credit risk

In [46]:
# % of missing values in one feature
print(loans.pub_rec_bankruptcies.value_counts(normalize=True, dropna=False))

 0.0    0.939438
 1.0    0.042456
NaN     0.017978
 2.0    0.000129
Name: pub_rec_bankruptcies, dtype: float64


* Nearly 94% of values are in the same category, it should be dropped since it has minimal predictive value

In [47]:
loans = loans.drop("pub_rec_bankruptcies", axis=1)

# Drop rows with missing value
loans = loans.dropna(axis=0)
print(loans.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


In [48]:
# Examine object columns

object_columns_df = loans.select_dtypes(include=["object"])
object_columns_df[:10]

,term,int_rate,emp_length,home_ownership,verification_status,purpose,title,addr_state,earliest_cr_line,revol_util,last_credit_pull_d
0,36 months,10.65%,10+ years,RENT,Verified,credit_card,Computer,AZ,Jan-1985,83.7%,Jun-2016
1,60 months,15.27%,< 1 year,RENT,Source Verified,car,bike,GA,Apr-1999,9.4%,Sep-2013
2,36 months,15.96%,10+ years,RENT,Not Verified,small_business,real estate business,IL,Nov-2001,98.5%,Jun-2016
3,36 months,13.49%,10+ years,RENT,Source Verified,other,personel,CA,Feb-1996,21%,Apr-2016
5,36 months,7.90%,3 years,RENT,Source Verified,wedding,My wedding loan I promise to pay back,AZ,Nov-2004,28.3%,Jan-2016
6,60 months,15.96%,8 years,RENT,Not Verified,debt_consolidation,Loan,NC,Jul-2005,85.6%,May-2016
7,36 months,18.64%,9 years,RENT,Source Verified,car,Car Downpayment,CA,Jan-2007,87.5%,Dec-2014
8,60 months,21.28%,4 years,OWN,Source Verified,small_business,Expand Business & Buy Debt Portfolio,CA,Apr-2004,32.6%,Aug-2012
9,60 months,12.69%,< 1 year,RENT,Verified,other,Building my credit history.,TX,Sep-2004,36.5%,Mar-2013
10,60 months,14.65%,5 years,OWN,Not Verified,debt_consolidation,High intrest Consolidation,AZ,Jan-1998,20.6%,Dec-2015


In [49]:
cols = ['title', 'purpose']
for c in cols:
    print(loans[c].value_counts())

Debt Consolidation                        2068
Debt Consolidation Loan                   1599
Personal Loan                              624
Consolidation                              488
debt consolidation                         466
Credit Card Consolidation                  345
Home Improvement                           336
Debt consolidation                         314
Small Business Loan                        298
Credit Card Loan                           294
Personal                                   290
Consolidation Loan                         250
Home Improvement Loan                      228
personal loan                              219
Loan                                       202
Wedding Loan                               199
personal                                   198
Car Loan                                   188
consolidation                              186
Other Loan                                 168
Wedding                                    148
Credit Card P

* Title column has too many discrete value
* "Title" and "Purpose" columns could reflect the same information so "Title" column can be dropped

In [50]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}


loans = loans.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)

loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype("float")
loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype("float")
loans = loans.replace(mapping_dict)

loans.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
0,5000.0,36 months,10.65,162.87,10,RENT,24000.0,Verified,1,credit_card,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0
1,2500.0,60 months,15.27,59.83,0,RENT,30000.0,Source Verified,0,car,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0
2,2400.0,36 months,15.96,84.33,10,RENT,12252.0,Not Verified,1,small_business,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0
3,10000.0,36 months,13.49,339.31,10,RENT,49200.0,Source Verified,1,other,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0
5,5000.0,36 months,7.90,156.46,3,RENT,36000.0,Source Verified,1,wedding,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0


In [56]:
cat_columns = ["home_ownership", "verification_status", "purpose", "term"]
dummy_df = pd.get_dummies(loans[cat_columns])
dummy_df.head()

,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,0,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,0,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
5,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [57]:
loans = pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(cat_columns, axis=1)

#### Error Metric

* False positive: predicting that a loan will be paid off on time, but it actually isn't 
    * => Losing Money
* False negative: predicting that a loan won't be paid off on time
    * => Missing out winners

==> Conservative investors would want to minimize risk (= avoid false positives as much as possible). 
* relatively fine with missing out on opportunities (= false negatives) than funding a risky loan (= false positives).

==> This means that we should optimize for:
* High recall (true positive rate)
* Low fall-out (false positive rate)

$ FPR=\dfrac{\text{False Positives}}{\text{False Positives} + \text{True Negatives}} $

False Positive Rate: the percentage of the loans that shouldn't be funded (= Out of all the Charged off (= 0) loans) that I would fund

$ TPR=\dfrac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}} $

True Positive Rate: the percentage of loans that should be funded (= Out of all the Fully Paid (= 1) loans)  that I would fund".


In [66]:
print(loans_2007['loan_status'].value_counts())

1    33136
0     5634
Name: loan_status, dtype: int64


In [67]:
loans = pd.read_csv("https://dsserver-prod-resources-1.s3.amazonaws.com/135/cleaned_loans_2007.csv?versionId=.k3f0ymk4OQ5mORyjlHErc9AzOYjErR6")

In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

lr = LogisticRegression()
predictions = cross_val_predict(lr, features, target, cv=3)

predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp  / (tp + fn)
fpr = fp  / (fp + tn)

print(tpr)
print(fpr)

C:\Users\Shane Kim\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Shane Kim\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Shane Kim\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9987920460880877
0.9962887363147152


Class imbalance between the positive and negative cases: 
* Fully paid loans are 6 times as many loans than Charged off loans.
* This will result in high accuracy in any model.
* The model ends up having a strong bias towards predicting the class with more observations in the training set and will rarely predict the class with less observations. The stronger the imbalance, the more biased the model becomes.

Set the "class_weight" parameter to "balanced" when creating the LogisticRegression instance.

This will penalize the misclassification of the minority class during the training process. The penalty means that the logistic regression classifier pays more attention to correctly classifying rows where loan_status is 0. This lowers accuracy when loan_status is 1, but raises accuracy when loan_status is 0.

The penalty is set to be inversely proportional to the class frequencies. It means that for the classifier, correctly classifying a row where loan_status is 0 is 6 times more important than correctly classifying a row where loan_status is 1.

In [69]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

lr = LogisticRegression(class_weight="balanced")

predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

C:\Users\Shane Kim\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Shane Kim\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Shane Kim\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6644675710834418
0.3859714232696233


Successfully reduced a False Positive rate to 38%, but true positive rate is also reduced to 66%

##### Misclassifying the negative class

Let's try to lower the false positive rate further by assigning a harsher penalty for misclassifying the negative class.

In the original data, the penalty for misclassifying a 0 would have been around 6.


In [70]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
penalty = {
    0: 10,
    1: 1
}

lr = LogisticRegression(class_weight=penalty)
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

C:\Users\Shane Kim\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Shane Kim\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Shane Kim\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.2514712259183547
0.09482278715902764


It lowered the false positive rate to 9% (lower default risk); however, true positive rate dropped too much to justify the model

In [74]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict

rf = RandomForestClassifier(class_weight="balanced", random_state=1)
predictions = cross_val_predict(rf, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.`
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

C:\Users\Shane Kim\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Shane Kim\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Shane Kim\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9700799107972495
0.9181666357394693


Unfortunately, Random forest classifier didn't improve our false positive rate

Ultimately, our best model had a false positive rate of nearly 9%, and a true positive rate of nearly 24%.
Interpretation: For a conservative investor, they can make money as long as the interest rate is high enough to offset the losses from 9% of borrowers defaulting, and that the pool of 24% of borrowers is large enough to make enough interest money to offset the losses.

* Appendix

In [28]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state', 'title', 'purpose']
for c in cols:
    print(loans[c].value_counts())

RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
 36 months    28234
 60 months     9441
Name: term, dtype: int64
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
AL     420
LA     420
KY     311
OK     285
KS     249
UT     249
AR     229
DC     209
RI     194
NM     180
WV     164
HI     162
NH     157
DE     110
MT      77
WY      76
AK      76
SD      60
VT  